# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
#for text pre-processing
import seaborn as sns
import re, string
import nltk
import time

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

# Importing all the 5 datasets

In [ ]:
df_rest=pd.read_csv("/content/drive/MyDrive/Phase 2 changes/Restaurant_Reviews.csv")

In [ ]:
df_airl=pd.read_csv("/content/drive/MyDrive/Phase 2 changes/Tweets.csv")

In [ ]:
df_fin=pd.read_csv("/content/drive/MyDrive/Phase 2 changes/financial.csv",encoding = "ISO-8859-1")

In [ ]:
df_fin_news=pd.read_csv("/content/drive/MyDrive/Phase 2 changes/sentiment_analysis_finnews.csv")

In [ ]:
df_gpt=pd.read_csv("/content/drive/MyDrive/Phase 2 changes/GPT.csv")

# EDA

In [ ]:
df_rest.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df_airl.head()

,airline_sentiment,text
0,1,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,1,@VirginAmerica I didn't today... Must mean I n...
3,0,@VirginAmerica it's really aggressive to blast...
4,0,@VirginAmerica and it's a really big bad thing...


In [ ]:
df_fin.head()

,setiment,text
0,1,"According to Gran , the company has no plans t..."
1,1,Technopolis plans to develop in stages an area...
2,0,The international electronic industry company ...
3,2,With the new production plant the company woul...
4,2,According to the company 's updated strategy f...


In [ ]:
df_fin_news.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,2
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",0
2,"For the last quarter of 2010 , Componenta 's n...",2
3,According to the Finnish-Russian Chamber of Co...,1
4,The Swedish buyout firm has sold its remaining...,1


In [ ]:
df_gpt.head()

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,1
1,"Try talking with ChatGPT, our new AI system wh...",2
2,ChatGPT: Optimizing Language Models for Dialog...,1
3,"THRILLED to share that ChatGPT, our new model ...",2
4,"As of 2 minutes ago, @OpenAI released their ne...",0


In [ ]:
df_rest_t=df_rest[:1000]

In [ ]:
df_rest_t.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df_airl_t=df_airl[:500]
df_fin_t=df_fin[:500]
df_fin_news_t=df_fin_news[:500]
df_gpt_t=df_gpt[:500]

In [ ]:
x1=df_rest_t['Liked'].value_counts()
print(x1)

1    500
0    500
Name: Liked, dtype: int64


In [ ]:
x2=df_airl_t['airline_sentiment'].value_counts()
print(x2)

0    181
1    168
2    151
Name: airline_sentiment, dtype: int64


# Processing the dataset using NLP techniques
##### Like removal of stopwords, Lemmatization, Tokenization

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [ ]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [ ]:
df_rest_t['clean_text_rest'] = df_rest_t['Review'].apply(lambda x: finalpreprocess(x))
df_rest_t.head()

<ipython-input-22-5915f68422c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rest_t['clean_text_rest'] = df_rest_t['Review'].apply(lambda x: finalpreprocess(x))


In [ ]:
df_rest_t

,Review,Liked,clean_text_rest
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust good
2,Not tasty and the texture was just nasty.,0,tasty texture nasty
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,selection menu great price
...,...,...,...
995,I think food should have flavor and texture an...,0,think food flavor texture lack
996,Appetite instantly gone.,0,appetite instantly go
997,Overall I was not impressed and would not go b...,0,overall impress would go back
998,"The whole experience was underwhelming, and I ...",0,whole experience underwhelming think go ninja ...


In [ ]:
df_airl_t['clean_text_airl'] = df_airl_t['text'].apply(lambda x: finalpreprocess(x))
df_airl_t.head()

df_fin_t['clean_text_fin'] = df_fin_t['text'].apply(lambda x: finalpreprocess(x))
df_fin_t.head()

df_fin_news_t['clean_text_fin_news'] = df_fin_news_t['Sentence'].apply(lambda x: finalpreprocess(x))
df_fin_news_t.head()


In [ ]:
df_gpt_t['clean_text_gpt'] = df_gpt_t['tweets'].apply(lambda x: finalpreprocess(x))
df_gpt_t.head()

<ipython-input-20-5ed089e60473>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gpt_t['clean_text_gpt'] = df_gpt_t['tweets'].apply(lambda x: finalpreprocess(x))


,tweets,labels,clean_text_gpt
0,ChatGPT: Optimizing Language Models for Dialog...,1,chatgpt optimize language model dialogue https...
1,"Try talking with ChatGPT, our new AI system wh...",2,try talk chatgpt new ai system optimize dialog...
2,ChatGPT: Optimizing Language Models for Dialog...,1,chatgpt optimize language model dialogue https...
3,"THRILLED to share that ChatGPT, our new model ...",2,thrill share chatgpt new model optimize dialog...
4,"As of 2 minutes ago, @OpenAI released their ne...",0,minute ago openai release new chatgpt n nand u...


# Splitting the datasets into Train-Test Datasets

In [ ]:
X_train_r , X_test_r, y_train_r, y_test_r = train_test_split(df_rest_t["clean_text_rest"],df_rest_t["Liked"],test_size=0.2,shuffle=True)

In [ ]:
X_train_a , X_test_a, y_train_a, y_test_a = train_test_split(df_airl_t["clean_text_airl"],df_airl_t["airline_sentiment"],test_size=0.2,shuffle=True)

In [ ]:
X_train_f , X_test_f, y_train_f, y_test_f = train_test_split(df_fin_t["clean_text_fin"],df_fin_t["setiment"],test_size=0.2,shuffle=True)

In [ ]:
X_train_fn , X_test_fn, y_train_fn, y_test_fn = train_test_split(df_fin_news_t["clean_text_fin_news"],df_fin_news_t["Sentiment"],test_size=0.2,shuffle=True)

In [ ]:
X_train_g , X_test_g, y_train_g, y_test_g = train_test_split(df_gpt_t["clean_text_gpt"],df_gpt_t["labels"],test_size=0.2,shuffle=True)

# Converting the text-data into vector form

In [ ]:
tfidf_vectorizer_r = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf_r = tfidf_vectorizer_r.fit_transform(X_train_r) 
X_test_vectors_tfidf_r = tfidf_vectorizer_r.transform(X_test_r)

In [ ]:
tfidf_vectorizer_a = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf_a = tfidf_vectorizer_a.fit_transform(X_train_a) 
X_test_vectors_tfidf_a = tfidf_vectorizer_a.transform(X_test_a)

In [ ]:
tfidf_vectorizer_f = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf_f = tfidf_vectorizer_f.fit_transform(X_train_f) 
X_test_vectors_tfidf_f = tfidf_vectorizer_f.transform(X_test_f)

In [ ]:
tfidf_vectorizer_fn = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf_fn = tfidf_vectorizer_fn.fit_transform(X_train_fn) 
X_test_vectors_tfidf_fn = tfidf_vectorizer_fn.transform(X_test_fn)

In [ ]:
tfidf_vectorizer_g = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf_g = tfidf_vectorizer_g.fit_transform(X_train_g) 
X_test_vectors_tfidf_g = tfidf_vectorizer_g.transform(X_test_g)

# CLASSIFICATION BEGINS

## Naive Bayes

<h4> Restaurant Dataset </h4>

In [ ]:
nb_tfidf_r = MultinomialNB()
start = time.time()
nb_tfidf_r.fit(X_train_vectors_tfidf_r, y_train_r)  
end = time.time()
#Predict y value for test dataset
y_predict = nb_tfidf_r.predict(X_test_vectors_tfidf_r)
y_prob = nb_tfidf_r.predict_proba(X_test_vectors_tfidf_r)[:,1]
score_nb = round(accuracy_score(y_predict,y_test_r)*100,2)
print("The accuracy score is: " + str(score_nb)) 
print(classification_report(y_test_r,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test_r, y_predict))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, y_predict,average='weighted'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r, y_predict, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r, y_predict, average='macro'))

# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

The accuracy score is: 72.0
              precision    recall  f1-score   support

           0       0.60      0.80      0.69        76
           1       0.85      0.67      0.75       124

    accuracy                           0.72       200
   macro avg       0.72      0.74      0.72       200
weighted avg       0.75      0.72      0.72       200

Confusion Matrix: [[61 15]
 [41 83]]
f1 score 0.7240530418058507
precision  0.7224889955982393
recall   0.7359932088285229


<h4> Airline Dataset </h4>

In [ ]:
nb_tfidf_a = MultinomialNB()
start = time.time()
nb_tfidf_a.fit(X_train_vectors_tfidf_a, y_train_a)  
end = time.time()
#Predict y value for test dataset
y_predict = nb_tfidf_a.predict(X_test_vectors_tfidf_a)
y_prob = nb_tfidf_a.predict_proba(X_test_vectors_tfidf_a)[:,1]
score_nb = round(accuracy_score(y_predict,y_test_a)*100,2)
print("The accuracy score is: " + str(score_nb)) 
print(classification_report(y_test_a,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test_a, y_predict))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, y_predict,average='weighted'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a, y_predict, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a, y_predict, average='macro'))

The accuracy score is: 63.0
              precision    recall  f1-score   support

           0       0.58      0.83      0.68        36
           1       0.70      0.53      0.60        40
           2       0.67      0.50      0.57        24

    accuracy                           0.63       100
   macro avg       0.65      0.62      0.62       100
weighted avg       0.65      0.63      0.62       100

Confusion Matrix: [[30  4  2]
 [15 21  4]
 [ 7  5 12]]
f1 score 0.6225974025974026
precision  0.6478632478632478
recall   0.6194444444444445


<h4> Financial Dataset </h4>

In [ ]:
nb_tfidf_f = MultinomialNB()
start = time.time()
nb_tfidf_f.fit(X_train_vectors_tfidf_f, y_train_f)  
end = time.time()
#Predict y value for test dataset
y_predict = nb_tfidf_f.predict(X_test_vectors_tfidf_f)
y_prob = nb_tfidf_f.predict_proba(X_test_vectors_tfidf_f)[:,1]
score_nb = round(accuracy_score(y_predict,y_test_f)*100,2)
print("The accuracy score is: " + str(score_nb)) 
print(classification_report(y_test_f,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test_f, y_predict))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, y_predict,average='weighted'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f, y_predict, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f, y_predict, average='macro'))

The accuracy score is: 86.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      0.13      0.24        15
           2       0.86      1.00      0.92        84

    accuracy                           0.86       100
   macro avg       0.62      0.38      0.39       100
weighted avg       0.87      0.86      0.81       100

Confusion Matrix: [[ 0  0  1]
 [ 0  2 13]
 [ 0  0 84]]
f1 score 0.8106787330316741
precision  0.6190476190476191
recall   0.37777777777777777


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> Financial News Datasets </h4>

In [ ]:
nb_tfidf_fn = MultinomialNB()
start = time.time()
nb_tfidf_fn.fit(X_train_vectors_tfidf_fn, y_train_fn)  
end = time.time()
#Predict y value for test dataset
y_predict = nb_tfidf_fn.predict(X_test_vectors_tfidf_fn)
y_prob = nb_tfidf_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]
score_nb = round(accuracy_score(y_predict,y_test_fn)*100,2)
print("The accuracy score is: " + str(score_nb)) 
print(classification_report(y_test_fn,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test_fn, y_predict))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, y_predict,average='weighted'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn, y_predict, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn, y_predict, average='macro'))

The accuracy score is: 51.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.50      1.00      0.67        48
           2       0.75      0.08      0.15        37

    accuracy                           0.51       100
   macro avg       0.42      0.36      0.27       100
weighted avg       0.52      0.51      0.37       100

Confusion Matrix: [[ 0 14  1]
 [ 0 48  0]
 [ 0 34  3]]
f1 score 0.37414634146341463
precision  0.4166666666666667
recall   0.3603603603603604


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> GPT Dataset </h4>

In [ ]:
nb_tfidf_g = MultinomialNB()
start = time.time()
nb_tfidf_g.fit(X_train_vectors_tfidf_g, y_train_g)  
end = time.time()
#Predict y value for test dataset
y_predict = nb_tfidf_g.predict(X_test_vectors_tfidf_g)
y_prob = nb_tfidf_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
score_nb = round(accuracy_score(y_predict,y_test_g)*100,2)
print("The accuracy score is: " + str(score_nb)) 
print(classification_report(y_test_g,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test_g, y_predict))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, y_predict,average='weighted'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g, y_predict, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g, y_predict, average='macro'))

The accuracy score is: 62.0
              precision    recall  f1-score   support

           0       0.71      0.71      0.71        45
           1       0.36      0.24      0.29        21
           2       0.61      0.74      0.67        34

    accuracy                           0.62       100
   macro avg       0.56      0.56      0.55       100
weighted avg       0.60      0.62      0.61       100

Confusion Matrix: [[32  5  8]
 [ 8  5  8]
 [ 5  4 25]]
f1 score 0.6066666666666667
precision  0.5593366886049812
recall   0.5615001556178026


## Random Forest

<h4> Restaurant Dataset </h4>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_accuracy = 0
for x in range(200):
  rf_r = RandomForestClassifier(random_state=x) 
  start = time.time()
  rf_r.fit(X_train_vectors_tfidf_r,y_train_r)
  end = time.time()
  Y_pred_rf = rf_r.predict(X_test_vectors_tfidf_r)
  current_accuracy = round(accuracy_score(Y_pred_rf,y_test_r)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x

In [ ]:
score_rf = round(accuracy_score (Y_pred_rf,y_test_r)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")
print(classification_report(y_test_r,Y_pred_rf))
print('Confusion Matrix:',confusion_matrix(y_test_r, Y_pred_rf))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, Y_pred_rf))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r, Y_pred_rf, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r, Y_pred_rf, average='macro'))

fpr, tpr, thresholds = roc_curve(y_test_r, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

The accuracy score achieved using Decision Tree is: 72.5 %
              precision    recall  f1-score   support

           0       0.60      0.84      0.70        76
           1       0.87      0.65      0.75       124

    accuracy                           0.73       200
   macro avg       0.73      0.75      0.72       200
weighted avg       0.77      0.72      0.73       200

Confusion Matrix: [[64 12]
 [43 81]]
f1 score 0.7465437788018433
precision  0.7345492915284896
recall   0.7476655348047538
AUC: 0.8197156196943974
The time of execution of above program is : 405.148983001709 ms


<h4> Airline Dataset </h4>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_accuracy = 0
for x in range(200):
  rf_a = RandomForestClassifier(random_state=x) 
  start = time.time()
  rf_a.fit(X_train_vectors_tfidf_a,y_train_a)
  end = time.time()
  Y_pred_rf = rf_a.predict(X_test_vectors_tfidf_a)
  current_accuracy = round(accuracy_score(Y_pred_rf,y_test_a)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
score_rf = round(accuracy_score (Y_pred_rf,y_test_a)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")
print(classification_report(y_test_a,Y_pred_rf))
print('Confusion Matrix:',confusion_matrix(y_test_a, Y_pred_rf))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, Y_pred_rf, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a, Y_pred_rf, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a, Y_pred_rf, average='macro'))

# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

(100,)
The accuracy score achieved using Decision Tree is: 58.0 %
              precision    recall  f1-score   support

           0       0.79      0.64      0.71        36
           1       0.51      0.70      0.59        40
           2       0.44      0.29      0.35        24

    accuracy                           0.58       100
   macro avg       0.58      0.54      0.55       100
weighted avg       0.59      0.58      0.57       100

Confusion Matrix: [[23 11  2]
 [ 5 28  7]
 [ 1 16  7]]
f1 score 0.5490553306342779
precision  0.5798981191222571
recall   0.5435185185185185


<h4> Financial Dataset </h4>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_accuracy = 0
for x in range(200):
  rf_f = RandomForestClassifier(random_state=x) 
  start = time.time()
  rf_f.fit(X_train_vectors_tfidf_f,y_train_f)
  end = time.time()
  Y_pred_rf = rf_f.predict(X_test_vectors_tfidf_f)
  current_accuracy = round(accuracy_score(Y_pred_rf,y_test_f)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x

score_rf = round(accuracy_score (Y_pred_rf,y_test_f)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")
print(classification_report(y_test_f,Y_pred_rf))
print('Confusion Matrix:',confusion_matrix(y_test_f, Y_pred_rf))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, Y_pred_rf, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f, Y_pred_rf, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f, Y_pred_rf, average='macro'))

The accuracy score achieved using Decision Tree is: 91.0 %
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.89      0.53      0.67        15
           2       0.91      0.99      0.95        84

    accuracy                           0.91       100
   macro avg       0.60      0.51      0.54       100
weighted avg       0.90      0.91      0.90       100

Confusion Matrix: [[ 0  0  1]
 [ 0  8  7]
 [ 0  1 83]]
f1 score 0.5384126984126985
precision  0.6003256003256002
recall   0.5071428571428571


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> Financial News Dataset </h4>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_accuracy = 0
for x in range(200):
  rf_fn = RandomForestClassifier(random_state=x) 
  start = time.time()
  rf_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
  end = time.time()
  Y_pred_rf = rf_fn.predict(X_test_vectors_tfidf_fn)
  current_accuracy = round(accuracy_score(Y_pred_rf,y_test_fn)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
rf_fn = RandomForestClassifier(random_state=best_x)
rf_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
Y_pred_rf = rf_fn.predict(X_test_vectors_tfidf_fn)
y_prob = rf_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]
print (Y_pred_rf.shape)
score_rf = round(accuracy_score (Y_pred_rf,y_test_fn)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")
print(classification_report(y_test_fn,Y_pred_rf))
print('Confusion Matrix:',confusion_matrix(y_test_fn, Y_pred_rf))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, Y_pred_rf, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn, Y_pred_rf, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn, Y_pred_rf, average='macro'))

(100,)
The accuracy score achieved using Decision Tree is: 60.0 %
              precision    recall  f1-score   support

           0       0.75      0.20      0.32        15
           1       0.55      0.98      0.71        48
           2       0.91      0.27      0.42        37

    accuracy                           0.60       100
   macro avg       0.74      0.48      0.48       100
weighted avg       0.71      0.60      0.54       100

Confusion Matrix: [[ 3 11  1]
 [ 1 47  0]
 [ 0 27 10]]
f1 score 0.4797410192147034
precision  0.7373440285204991
recall   0.4831456456456456


<h4> GPT Dataset </h4>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
max_accuracy = 0
for x in range(200):
  rf_g = RandomForestClassifier(random_state=x) 
  start = time.time()
  rf_g.fit(X_train_vectors_tfidf_g,y_train_g)
  end = time.time()
  Y_pred_rf = rf_g.predict(X_test_vectors_tfidf_g)
  current_accuracy = round(accuracy_score(Y_pred_rf,y_test_g)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
rf_g = RandomForestClassifier(random_state=best_x)
rf_g.fit(X_train_vectors_tfidf_g,y_train_g)
Y_pred_rf = rf_g.predict(X_test_vectors_tfidf_g)
y_prob = rf_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
# print (Y_pred_rf.shape)
score_rf = round(accuracy_score (Y_pred_rf,y_test_g)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")
print(classification_report(y_test_g,Y_pred_rf))
print('Confusion Matrix:',confusion_matrix(y_test_g, Y_pred_rf))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, Y_pred_rf, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g, Y_pred_rf, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g, Y_pred_rf, average='macro'))

The accuracy score achieved using Decision Tree is: 66.0 %
              precision    recall  f1-score   support

           0       0.64      0.91      0.75        45
           1       0.70      0.33      0.45        21
           2       0.69      0.53      0.60        34

    accuracy                           0.66       100
   macro avg       0.68      0.59      0.60       100
weighted avg       0.67      0.66      0.64       100

Confusion Matrix: [[41  0  4]
 [10  7  4]
 [13  3 18]]
f1 score 0.6013021604024859
precision  0.6776442307692307
recall   0.5912854030501089


## Logistic Regression

<h4> Restaurant Dataset </h4>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_r = LogisticRegression()
start = time.time()
lr_r.fit(X_train_vectors_tfidf_r,y_train_r)
end = time.time()
Y_pred_lr = lr_r.predict(X_test_vectors_tfidf_r)
y_prob = lr_r.predict_proba(X_test_vectors_tfidf_r)[:,1]

In [ ]:
score_lr = round(accuracy_score(Y_pred_lr,y_test_r)*100,2)
print("The accuracy score is: " + str(score_lr))
print(classification_report(y_test_r,Y_pred_lr))
print('Confusion Matrix:',confusion_matrix(y_test_r, Y_pred_lr))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, Y_pred_lr))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r,  Y_pred_lr, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r,  Y_pred_lr, average='macro'))

# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")


The accuracy score is: 72.0
              precision    recall  f1-score   support

           0       0.59      0.87      0.70        76
           1       0.89      0.63      0.74       124

    accuracy                           0.72       200
   macro avg       0.74      0.75      0.72       200
weighted avg       0.77      0.72      0.72       200

Confusion Matrix: [[66 10]
 [46 78]]
f1 score 0.7358490566037735
precision  0.7378246753246753
recall   0.7487266553480476


<h4> Airline Dataset </h4>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_a = LogisticRegression()
start = time.time()
lr_a.fit(X_train_vectors_tfidf_a,y_train_a)
end = time.time()
Y_pred_lr = lr_a.predict(X_test_vectors_tfidf_a)
y_prob = lr_a.predict_proba(X_test_vectors_tfidf_a)[:,1]
# Y_pred_lr.shape
score_lr = round(accuracy_score(Y_pred_lr,y_test_a)*100,2)
print("The accuracy score is: " + str(score_lr))
print(classification_report(y_test_a,Y_pred_lr))
print('Confusion Matrix:',confusion_matrix(y_test_a, Y_pred_lr))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, Y_pred_lr, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a,  Y_pred_lr, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a,  Y_pred_lr, average='macro'))

The accuracy score is: 63.0
              precision    recall  f1-score   support

           0       0.64      0.78      0.70        36
           1       0.62      0.60      0.61        40
           2       0.65      0.46      0.54        24

    accuracy                           0.63       100
   macro avg       0.63      0.61      0.61       100
weighted avg       0.63      0.63      0.62       100

Confusion Matrix: [[28  6  2]
 [12 24  4]
 [ 4  9 11]]
f1 score 0.6147267675208398
precision  0.6329356917592212
recall   0.612037037037037


<h4> Financial Dataset </h4>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_f = LogisticRegression()
start = time.time()
lr_f.fit(X_train_vectors_tfidf_f,y_train_f)
end = time.time()
Y_pred_lr = lr_f.predict(X_test_vectors_tfidf_f)
y_prob = lr_f.predict_proba(X_test_vectors_tfidf_f)[:,1]

score_lr = round(accuracy_score(Y_pred_lr,y_test_f)*100,2)
print("The accuracy score is: " + str(score_lr))
print(classification_report(y_test_f,Y_pred_lr))
print('Confusion Matrix:',confusion_matrix(y_test_f, Y_pred_lr))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, Y_pred_lr, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f,  Y_pred_lr, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f,  Y_pred_lr, average='macro'))

The accuracy score is: 87.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.80      0.27      0.40        15
           2       0.87      0.99      0.93        84

    accuracy                           0.87       100
   macro avg       0.56      0.42      0.44       100
weighted avg       0.85      0.87      0.84       100

Confusion Matrix: [[ 0  0  1]
 [ 0  4 11]
 [ 0  1 83]]
f1 score 0.44245810055865925
precision  0.5578947368421053
recall   0.41825396825396827


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> Financial News Dataset </h4>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_fn = LogisticRegression()
start = time.time()
lr_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
end = time.time()
Y_pred_lr = lr_fn.predict(X_test_vectors_tfidf_fn)
y_prob = lr_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]

score_lr = round(accuracy_score(Y_pred_lr,y_test_fn)*100,2)
print("The accuracy score is: " + str(score_lr))
print(classification_report(y_test_fn,Y_pred_lr))
print('Confusion Matrix:',confusion_matrix(y_test_fn, Y_pred_lr))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, Y_pred_lr, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn,  Y_pred_lr, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn,  Y_pred_lr, average='macro'))

The accuracy score is: 53.0
              precision    recall  f1-score   support

           0       1.00      0.07      0.12        15
           1       0.51      0.98      0.67        48
           2       0.71      0.14      0.23        37

    accuracy                           0.53       100
   macro avg       0.74      0.39      0.34       100
weighted avg       0.66      0.53      0.43       100

Confusion Matrix: [[ 1 13  1]
 [ 0 47  1]
 [ 0 32  5]]
f1 score 0.34123376623376617
precision  0.7417184265010351
recall   0.3936561561561562


<h4> GPT Dataset </h4>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_g = LogisticRegression()
start = time.time()
lr_g.fit(X_train_vectors_tfidf_g,y_train_g)
end = time.time()
Y_pred_lr = lr_g.predict(X_test_vectors_tfidf_g)
y_prob = lr_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
# Y_pred_lr.shape
score_lr = round(accuracy_score(Y_pred_lr,y_test_g)*100,2)
print("The accuracy score is: " + str(score_lr))
print(classification_report(y_test_g,Y_pred_lr))
print('Confusion Matrix:',confusion_matrix(y_test_g, Y_pred_lr))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, Y_pred_lr, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g,  Y_pred_lr, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g,  Y_pred_lr, average='macro'))

The accuracy score is: 61.0
              precision    recall  f1-score   support

           0       0.70      0.71      0.70        45
           1       0.38      0.24      0.29        21
           2       0.59      0.71      0.64        34

    accuracy                           0.61       100
   macro avg       0.56      0.55      0.55       100
weighted avg       0.59      0.61      0.60       100

Confusion Matrix: [[32  3 10]
 [ 9  5  7]
 [ 5  5 24]]
f1 score 0.5458047834518422
precision  0.5552111373956549
recall   0.5516962340491752


## Decision Tree

<h4> Restaurant Dataset </h4>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
max_accuracy = 0
for x in range(200):
  dt_r = DecisionTreeClassifier(random_state=x) 
  start = time.time()

  dt_r.fit(X_train_vectors_tfidf_r,y_train_r)
  end = time.time()
  Y_pred_dt = dt_r.predict(X_test_vectors_tfidf_r)
  current_accuracy = round(accuracy_score (Y_pred_dt,y_test_r)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x

In [ ]:
score_dt = round(accuracy_score (Y_pred_dt,y_test_r)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(classification_report(y_test_r,Y_pred_dt))
print('Confusion Matrix:',confusion_matrix(y_test_r, Y_pred_dt))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, Y_pred_dt, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r, Y_pred_dt, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r, Y_pred_dt, average='macro'))

# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

The accuracy score achieved using Decision Tree is: 73.0 %
              precision    recall  f1-score   support

           0       0.62      0.76      0.68        76
           1       0.83      0.71      0.77       124

    accuracy                           0.73       200
   macro avg       0.72      0.74      0.72       200
weighted avg       0.75      0.73      0.73       200

Confusion Matrix: [[58 18]
 [36 88]]
f1 score 0.7237851662404092
precision  0.7236049779205138
recall   0.7364176570458405


<h4> Airline Dataset </h4>

In [ ]:
max_accuracy = 0
for x in range(200):
  dt_a = DecisionTreeClassifier(random_state=x) 
  start = time.time()

  dt_a.fit(X_train_vectors_tfidf_a,y_train_a)
  end = time.time()
  Y_pred_dt = dt_a.predict(X_test_vectors_tfidf_a)
  current_accuracy = round(accuracy_score (Y_pred_dt,y_test_a)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x

score_dt = round(accuracy_score (Y_pred_dt,y_test_a)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(classification_report(y_test_a,Y_pred_dt))
print('Confusion Matrix:',confusion_matrix(y_test_a, Y_pred_dt))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, Y_pred_dt, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a, Y_pred_dt, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a, Y_pred_dt, average='macro'))

(100,)
The accuracy score achieved using Decision Tree is: 52.0 %
              precision    recall  f1-score   support

           0       0.70      0.39      0.50        36
           1       0.48      0.70      0.57        40
           2       0.45      0.42      0.43        24

    accuracy                           0.52       100
   macro avg       0.55      0.50      0.50       100
weighted avg       0.55      0.52      0.51       100

Confusion Matrix: [[14 17  5]
 [ 5 28  7]
 [ 1 13 10]]
f1 score 0.5020703933747411
precision  0.54576802507837
recall   0.5018518518518519


<h4> Financial Dataset </h4>

In [ ]:
max_accuracy = 0
for x in range(200):
  dt_f = DecisionTreeClassifier(random_state=x) 
  start = time.time()

  dt_f.fit(X_train_vectors_tfidf_f,y_train_f)
  end = time.time()
  Y_pred_dt = dt_f.predict(X_test_vectors_tfidf_f)
  current_accuracy = round(accuracy_score (Y_pred_dt,y_test_f)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
dt_f = DecisionTreeClassifier(random_state=best_x)
dt_f.fit(X_train_vectors_tfidf_f,y_train_f)
Y_pred_dt = dt_f.predict(X_test_vectors_tfidf_f)
y_prob = dt_f.predict_proba(X_test_vectors_tfidf_f)[:,1]
# print (Y_pred_dt.shape)
score_dt = round(accuracy_score (Y_pred_dt,y_test_f)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(classification_report(y_test_f,Y_pred_dt))
print('Confusion Matrix:',confusion_matrix(y_test_f, Y_pred_dt))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, Y_pred_dt, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f, Y_pred_dt, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f, Y_pred_dt, average='macro'))

The accuracy score achieved using Decision Tree is: 86.0 %
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.54      0.47      0.50        15
           2       0.91      0.93      0.92        84

    accuracy                           0.86       100
   macro avg       0.82      0.80      0.81       100
weighted avg       0.85      0.86      0.86       100

Confusion Matrix: [[ 1  0  0]
 [ 0  7  8]
 [ 0  6 78]]
f1 score 0.8058823529411764
precision  0.8151460942158616
recall   0.7984126984126986


<h4> Financial News Dataset </h4>

In [ ]:
max_accuracy = 0
for x in range(200):
  dt_fn = DecisionTreeClassifier(random_state=x) 
  start = time.time()

  dt_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
  end = time.time()
  Y_pred_dt = dt_fn.predict(X_test_vectors_tfidf_fn)
  current_accuracy = round(accuracy_score (Y_pred_dt,y_test_fn)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
dt_fn = DecisionTreeClassifier(random_state=best_x)
dt_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
Y_pred_dt = dt_fn.predict(X_test_vectors_tfidf_fn)
y_prob = dt_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]
# print (Y_pred_dt.shape)
score_dt = round(accuracy_score (Y_pred_dt,y_test_fn)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(classification_report(y_test_fn,Y_pred_dt))
print('Confusion Matrix:',confusion_matrix(y_test_fn, Y_pred_dt))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, Y_pred_dt, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn, Y_pred_dt, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn, Y_pred_dt, average='macro'))

The accuracy score achieved using Decision Tree is: 51.0 %
              precision    recall  f1-score   support

           0       0.18      0.20      0.19        15
           1       0.57      0.77      0.65        48
           2       0.61      0.30      0.40        37

    accuracy                           0.51       100
   macro avg       0.45      0.42      0.41       100
weighted avg       0.53      0.51      0.49       100

Confusion Matrix: [[ 3  9  3]
 [ 7 37  4]
 [ 7 19 11]]
f1 score 0.41412241887905604
precision  0.4522708228590582
recall   0.42271021021021027


<h4> GPT Dataset </h4>

In [ ]:
max_accuracy = 0
for x in range(200):
  dt_g = DecisionTreeClassifier(random_state=x) 
  start = time.time()

  dt_g.fit(X_train_vectors_tfidf_g,y_train_g)
  end = time.time()
  Y_pred_dt = dt_g.predict(X_test_vectors_tfidf_g)
  current_accuracy = round(accuracy_score (Y_pred_dt,y_test_g)*100,2)
  if(current_accuracy>max_accuracy): 
    max_accuracy = current_accuracy 
    best_x = x
dt_g = DecisionTreeClassifier(random_state=best_x)
dt_g.fit(X_train_vectors_tfidf_g,y_train_g)
Y_pred_dt = dt_g.predict(X_test_vectors_tfidf_g)
y_prob = dt_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
# print (Y_pred_dt.shape)
score_dt = round(accuracy_score (Y_pred_dt,y_test_g)*100,2)
print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")
print(classification_report(y_test_g,Y_pred_dt))
print('Confusion Matrix:',confusion_matrix(y_test_g, Y_pred_dt))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, Y_pred_dt, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g, Y_pred_dt, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g, Y_pred_dt, average='macro'))

The accuracy score achieved using Decision Tree is: 57.0 %
              precision    recall  f1-score   support

           0       0.67      0.80      0.73        45
           1       0.32      0.29      0.30        21
           2       0.56      0.44      0.49        34

    accuracy                           0.57       100
   macro avg       0.51      0.51      0.51       100
weighted avg       0.56      0.57      0.56       100

Confusion Matrix: [[36  4  5]
 [ 8  6  7]
 [10  9 15]]
f1 score 0.5063586686537506
precision  0.5126705653021443
recall   0.5089635854341737


## SVM

<h4> Restaurant Dataset </h4>

In [ ]:
from sklearn import svm
sv_r=svm.SVC(kernel='linear',probability=True)
start = time.time()
sv_r.fit(X_train_vectors_tfidf_r,y_train_r)
end = time.time()
Y_pred_svm = sv_r.predict(X_test_vectors_tfidf_r) 
y_prob = sv_r.predict_proba(X_test_vectors_tfidf_r)[:,1]

In [ ]:
score_svm = round(accuracy_score(Y_pred_svm,y_test_r)*100,2)
print("The accuracy score is: " + str(score_svm))
print(classification_report(y_test_r,Y_pred_svm))
print('Confusion Matrix:',confusion_matrix(y_test_r, Y_pred_svm))


from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, Y_pred_svm, average='macro'))


from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r, Y_pred_svm, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r, Y_pred_svm, average='macro'))

# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

The accuracy score is: 74.5
              precision    recall  f1-score   support

           0       0.62      0.86      0.72        76
           1       0.88      0.68      0.77       124

    accuracy                           0.74       200
   macro avg       0.75      0.77      0.74       200
weighted avg       0.78      0.74      0.75       200

Confusion Matrix: [[65 11]
 [40 84]]
f1 score 0.742677665935064
precision  0.7516290726817043
recall   0.7663412563667232


<h4> Airline Dataset </h4>

In [ ]:
from sklearn import svm
sv_a=svm.SVC(kernel='linear',probability=True)
start = time.time()
sv_a.fit(X_train_vectors_tfidf_a,y_train_a)
end = time.time()
Y_pred_svm = sv_a.predict(X_test_vectors_tfidf_a) 
y_prob = sv_a.predict_proba(X_test_vectors_tfidf_a)[:,1]
score_svm = round(accuracy_score(Y_pred_svm,y_test_a)*100,2)
print("The accuracy score is: " + str(score_svm))
print(classification_report(y_test_a,Y_pred_svm))
print('Confusion Matrix:',confusion_matrix(y_test_a, Y_pred_svm))


from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, Y_pred_svm, average='macro'))


from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a, Y_pred_svm, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a, Y_pred_svm, average='macro'))

The accuracy score is: 61.0
              precision    recall  f1-score   support

           0       0.66      0.69      0.68        36
           1       0.58      0.65      0.61        40
           2       0.59      0.42      0.49        24

    accuracy                           0.61       100
   macro avg       0.61      0.59      0.59       100
weighted avg       0.61      0.61      0.61       100

Confusion Matrix: [[25  8  3]
 [10 26  4]
 [ 3 11 10]]
f1 score 0.5917484198689363
precision  0.6079692695791766
recall   0.5870370370370371


<h4> Financial Dataset </h4>

In [ ]:
from sklearn import svm
sv_f=svm.SVC(kernel='linear',probability=True)
start = time.time()
sv_f.fit(X_train_vectors_tfidf_f,y_train_f)
end = time.time()
Y_pred_svm = sv_f.predict(X_test_vectors_tfidf_f) 
y_prob = sv_f.predict_proba(X_test_vectors_tfidf_f)[:,1]
score_svm = round(accuracy_score(Y_pred_svm,y_test_f)*100,2)
print("The accuracy score is: " + str(score_svm))
print(classification_report(y_test_f,Y_pred_svm))
print('Confusion Matrix:',confusion_matrix(y_test_f, Y_pred_svm))


from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, Y_pred_svm, average='macro'))


from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f, Y_pred_svm, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f, Y_pred_svm, average='macro'))

The accuracy score is: 88.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.70      0.47      0.56        15
           2       0.90      0.96      0.93        84

    accuracy                           0.88       100
   macro avg       0.53      0.48      0.50       100
weighted avg       0.86      0.88      0.87       100

Confusion Matrix: [[ 0  0  1]
 [ 0  7  8]
 [ 0  3 81]]
f1 score 0.49701149425287355
precision  0.5333333333333333
recall   0.476984126984127


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> Financial News Dataset </h4>

In [ ]:
from sklearn import svm
sv_fn=svm.SVC(kernel='linear',probability=True)
start = time.time()
sv_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
end = time.time()
Y_pred_svm = sv_fn.predict(X_test_vectors_tfidf_fn) 
y_prob = sv_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]
score_svm = round(accuracy_score(Y_pred_svm,y_test_fn)*100,2)
print("The accuracy score is: " + str(score_svm))
print(classification_report(y_test_fn,Y_pred_svm))
print('Confusion Matrix:',confusion_matrix(y_test_fn, Y_pred_svm))


from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, Y_pred_svm, average='macro'))


from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn, Y_pred_svm, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn, Y_pred_svm, average='macro'))

The accuracy score is: 58.0
              precision    recall  f1-score   support

           0       0.60      0.20      0.30        15
           1       0.55      0.96      0.70        48
           2       0.82      0.24      0.38        37

    accuracy                           0.58       100
   macro avg       0.66      0.47      0.46       100
weighted avg       0.66      0.58      0.52       100

Confusion Matrix: [[ 3 11  1]
 [ 1 46  1]
 [ 1 27  9]]
f1 score 0.45732323232323235
precision  0.6552669552669553
recall   0.4671921921921922


<h4> GPT Dataset </h4>

In [ ]:
from sklearn import svm
sv_g=svm.SVC(kernel='linear',probability=True)
start = time.time()
sv_g.fit(X_train_vectors_tfidf_g,y_train_g)
end = time.time()
Y_pred_svm = sv_g.predict(X_test_vectors_tfidf_g) 
y_prob = sv_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
score_svm = round(accuracy_score(Y_pred_svm,y_test_g)*100,2)
print("The accuracy score is: " + str(score_svm))
print(classification_report(y_test_g,Y_pred_svm))
print('Confusion Matrix:',confusion_matrix(y_test_g, Y_pred_svm))


from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, Y_pred_svm, average='macro'))


from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g, Y_pred_svm, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g, Y_pred_svm, average='macro'))

The accuracy score is: 57.0
              precision    recall  f1-score   support

           0       0.68      0.71      0.70        45
           1       0.38      0.29      0.32        21
           2       0.51      0.56      0.54        34

    accuracy                           0.57       100
   macro avg       0.52      0.52      0.52       100
weighted avg       0.56      0.57      0.56       100

Confusion Matrix: [[32  3 10]
 [ 7  6  8]
 [ 8  7 19]]
f1 score 0.5183959219476671
precision  0.5231215257811003
recall   0.5185496420790539


## KNN

<h4> Restaurant Dataset </h4>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_r = KNeighborsClassifier(n_neighbors=7)
start = time.time()
knn_r.fit(X_train_vectors_tfidf_r,y_train_r)
end = time.time()

Y_pred_knn=knn_r.predict(X_test_vectors_tfidf_r)
Y_pred_knn.shape
score_knn = round(accuracy_score (Y_pred_knn,y_test_r)*100,2)

y_prob = knn_r.predict_proba(X_test_vectors_tfidf_r)[:,1]
print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")
print(classification_report(y_test_r,Y_pred_knn))
print('Confusion Matrix:',confusion_matrix(y_test_r, Y_pred_knn))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_r, Y_pred_knn))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_r, Y_pred_knn, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_r, Y_pred_knn, average='macro'))

# fpr, tpr, thresholds = roc_curve(y_test_f, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)
# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

The accuracy score achieved using KNN is: 73.0 %
              precision    recall  f1-score   support

           0       0.62      0.72      0.67        76
           1       0.81      0.73      0.77       124

    accuracy                           0.73       200
   macro avg       0.72      0.73      0.72       200
weighted avg       0.74      0.73      0.73       200

Confusion Matrix: [[55 21]
 [33 91]]
f1 score 0.7711864406779662
precision  0.71875
recall   0.7287775891341257


<h4> Airline Dataset </h4>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_a = KNeighborsClassifier(n_neighbors=7)
start = time.time()
knn_a.fit(X_train_vectors_tfidf_a,y_train_a)
end = time.time()

Y_pred_knn=knn_a.predict(X_test_vectors_tfidf_a)
Y_pred_knn.shape
score_knn = round(accuracy_score (Y_pred_knn,y_test_a)*100,2)

y_prob = knn_a.predict_proba(X_test_vectors_tfidf_a)[:,1]
print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")
print(classification_report(y_test_a,Y_pred_knn))
print('Confusion Matrix:',confusion_matrix(y_test_a, Y_pred_knn))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_a, Y_pred_knn, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_a, Y_pred_knn, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_a, Y_pred_knn, average='macro'))

The accuracy score achieved using KNN is: 61.0 %
              precision    recall  f1-score   support

           0       0.59      0.67      0.62        36
           1       0.63      0.60      0.62        40
           2       0.62      0.54      0.58        24

    accuracy                           0.61       100
   macro avg       0.61      0.60      0.61       100
weighted avg       0.61      0.61      0.61       100

Confusion Matrix: [[24  9  3]
 [11 24  5]
 [ 6  5 13]]
f1 score 0.6055130055130055
precision  0.6119974733581922
recall   0.6027777777777777


<h4> Financial Dataset </h4>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_f = KNeighborsClassifier(n_neighbors=7)
start = time.time()
knn_f.fit(X_train_vectors_tfidf_f,y_train_f)
end = time.time()

Y_pred_knn=knn_f.predict(X_test_vectors_tfidf_f)
Y_pred_knn.shape
score_knn = round(accuracy_score (Y_pred_knn,y_test_f)*100,2)

y_prob = knn_f.predict_proba(X_test_vectors_tfidf_f)[:,1]
print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")
print(classification_report(y_test_f,Y_pred_knn))
print('Confusion Matrix:',confusion_matrix(y_test_f, Y_pred_knn))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_f, Y_pred_knn, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_f, Y_pred_knn, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_f, Y_pred_knn, average='macro'))

The accuracy score achieved using KNN is: 86.0 %
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.57      0.53      0.55        15
           2       0.91      0.93      0.92        84

    accuracy                           0.86       100
   macro avg       0.49      0.49      0.49       100
weighted avg       0.85      0.86      0.85       100

Confusion Matrix: [[ 0  0  1]
 [ 0  8  7]
 [ 0  6 78]]
f1 score 0.48979039891818793
precision  0.49280177187153934
recall   0.48730158730158735


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

<h4> Financial News Dataset </h4>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_fn = KNeighborsClassifier(n_neighbors=7)
start = time.time()
knn_fn.fit(X_train_vectors_tfidf_fn,y_train_fn)
end = time.time()

Y_pred_knn=knn_fn.predict(X_test_vectors_tfidf_fn)
Y_pred_knn.shape
score_knn = round(accuracy_score (Y_pred_knn,y_test_fn)*100,2)

y_prob = knn_fn.predict_proba(X_test_vectors_tfidf_fn)[:,1]
print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")
print(classification_report(y_test_fn,Y_pred_knn))
print('Confusion Matrix:',confusion_matrix(y_test_fn, Y_pred_knn))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_fn, Y_pred_knn, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_fn, Y_pred_knn, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_fn, Y_pred_knn, average='macro'))

The accuracy score achieved using KNN is: 66.0 %
              precision    recall  f1-score   support

           0       0.30      0.30      0.30        10
           1       0.73      0.84      0.78        63
           2       0.59      0.37      0.45        27

    accuracy                           0.66       100
   macro avg       0.54      0.50      0.51       100
weighted avg       0.65      0.66      0.64       100

Confusion Matrix: [[ 3  7  0]
 [ 3 53  7]
 [ 4 13 10]]
f1 score 0.511319073083779
precision  0.5380875637926404
recall   0.5038800705467371


<h4> GPT Dataset </h4>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_g = KNeighborsClassifier(n_neighbors=7)
start = time.time()
knn_g.fit(X_train_vectors_tfidf_g,y_train_g)
end = time.time()

Y_pred_knn=knn_g.predict(X_test_vectors_tfidf_g)
Y_pred_knn.shape
score_knn = round(accuracy_score (Y_pred_knn,y_test_g)*100,2)

y_prob = knn_g.predict_proba(X_test_vectors_tfidf_g)[:,1]
print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")
print(classification_report(y_test_g,Y_pred_knn))
print('Confusion Matrix:',confusion_matrix(y_test_g, Y_pred_knn))

from sklearn.metrics import f1_score
print("f1 score",f1_score(y_test_g, Y_pred_knn, average='macro'))

from sklearn.metrics import precision_score
print("precision ",precision_score(y_test_g, Y_pred_knn, average='macro'))

from sklearn.metrics import recall_score
print("recall  ",recall_score(y_test_g, Y_pred_knn, average='macro'))

The accuracy score achieved using KNN is: 54.0 %
              precision    recall  f1-score   support

           0       0.69      0.53      0.60        45
           1       0.48      0.48      0.48        29
           2       0.44      0.62      0.52        26

    accuracy                           0.54       100
   macro avg       0.54      0.54      0.53       100
weighted avg       0.56      0.54      0.54       100

Confusion Matrix: [[24 10 11]
 [ 6 14  9]
 [ 5  5 16]]
f1 score 0.5329625509825733
precision  0.5376391169494618
recall   0.543825523135868


# Exporting the data to the CSV files

In [ ]:
final_r=df_rest_t[['clean_text_rest','Liked']].reset_index(drop=True)

In [ ]:
final_r

,clean_text_rest,Liked
0,wow love place,1
1,crust good,0
2,tasty texture nasty,0
3,stop late may bank holiday rick steve recommen...,1
4,selection menu great price,1
...,...,...
995,think food flavor texture lack,0
996,appetite instantly go,0
997,overall impress would go back,0
998,whole experience underwhelming think go ninja ...,0


In [ ]:
final_r.to_csv('/content/drive/MyDrive/Phase 2 changes/clean_data_r.csv')

In [ ]:
final_a=df_airl_t[['clean_text_airl','airline_sentiment']].reset_index(drop=True)
final_a

,clean_text_airl,airline_sentiment
0,virginamerica dhepburn say,1
1,virginamerica plus added commercial experience...,2
2,virginamerica today must mean need take anothe...,1
3,virginamerica really aggressive blast obnoxiou...,0
4,virginamerica really big bad thing,0
...,...,...
495,virginamerica flight schedule pm departure sti...,0
496,virginamerica group get cancel flightlation fe...,0
497,virginamerica sign bonuses enroll elevate,1
498,virginamerica get pm newark home la folk ewr i...,2


In [ ]:
final_a.to_csv('/content/drive/MyDrive/Phase 2 changes/clean_data_a.csv')

In [ ]:
final_f=df_fin_t[['clean_text_fin','setiment']].reset_index(drop=True)
final_f

,clean_text_fin,setiment
0,accord gran company plan move production russi...,1
1,technopolis plan develop stage area less squar...,1
2,international electronic industry company elco...,0
3,new production plant company would increase ca...,2
4,accord company update strategy year basware ta...,2
...,...,...
495,delivers different user experience enable u wi...,2
496,jacob support storengy investment develop stor...,1
497,adpnews jun finnish stainless steel maker outo...,2
498,increase net sale due acquisition swedish silv...,2


In [ ]:
final_f.to_csv('/content/drive/MyDrive/Phase 2 changes/clean_data_f.csv')

In [ ]:
final_fn=df_fin_news_t[['clean_text_fin_news','Sentiment']].reset_index(drop=True)
final_fn

,clean_text_fin_news,Sentiment
0,geosolutions technology leverage benefon gps s...,2
1,esi low bk real possibility,0
2,last quarter componenta net sale double eur eu...,2
3,accord finnish russian chamber commerce major ...,1
4,swedish buyout firm sell remain percent stake ...,1
...,...,...
495,february new word identity document act make m...,1
496,antti orkola president kemira growhow industri...,0
497,alone,1
498,february finland base steel maker rautaruukki ...,0


In [ ]:
final_fn.to_csv('/content/drive/MyDrive/Phase 2 changes/clean_data_fn.csv')

In [ ]:
final_g=df_gpt_t[['clean_text_gpt','labels']].reset_index(drop=True)
final_g

,clean_text_gpt,labels
0,chatgpt optimize language model dialogue https...,1
1,try talk chatgpt new ai system optimize dialog...,2
2,chatgpt optimize language model dialogue https...,1
3,thrill share chatgpt new model optimize dialog...,2
4,minute ago openai release new chatgpt n nand u...,0
...,...,...
495,ok sorry crazy chatgpt massive education alrea...,2
496,glad hear chatgpt n n rsna radiology ai http c...,1
497,openai chatgpt basically jarvis level n nit im...,2
498,see enough chatgpt beanie baby profit dm inves...,1


In [ ]:
final_g.to_csv('/content/drive/MyDrive/Phase 2 changes/clean_data_g.csv')